# Using Jurassic-2 Mid on SageMaker through Model Packages

This sample notebook shows you how to deploy **Jurassic-2 Mid** (formerly Grande Instruct) using Amazon SageMaker.

Jurassic-2 Mid is a best-in-class large language model (LLM) by AI21 Labs. As a mid-sized model, it is carefully designed to strike the right balance between exceptional quality and affordability. Apply Jurassic-2 Mid to any language comprehension or generation task such as question answering, summarization, generating long-form copy, advanced information extraction and many others. It also supports non-English languages including Spanish, French, German, Portuguese, Italian and Dutch.


## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
1. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   1. [Interact with the model](#B.-Interact-with-the-model)
   1. [Prompt with instructions](#C.-Prompt-with-instructions)
   1. [Prompt with examples](#D.-Prompt-with-examples)
1. [Clean-up](#3.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   1. [Delete the model](#B.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [1]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/j2-mid-v1-1-053-8a8f3ed1f4093117ab66a2f52de4d2a9"
}

In [2]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [3]:
boto3.__version__

'1.26.74'

### Install ai21 python SDK

In [4]:
! pip install -U "ai21[AWS]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [5]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [6]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

### <span style='color:Blue'> How to choose the best instance for my use case?</span>
<span style='color:#0057FF'> When you create your endpoint, you need to choose the instance type to run the model on. Choosing the right instance is mainly a matter of economics. Depending on your use case, you probably want the most cost-effective instance possible. In this notebook we use one of the supported instances.</span>

<span style='color:#0057FF'>Looking for the list of all supported instances? See</span> [here](https://docs.ai21.com/docs/choosing-the-right-instance-type-for-amazon-sagemaker-models#jurassic-2-mid).

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [7]:
endpoint_name = "j2-mid"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.48xlarge"    # Recommended instance
#     "ml.g5.12xlarge"   # Cheaper and faster - recommended for relatively short inputs/outputs
#     "ml.p4d.24xlarge" # Recommended for long inputs/outputs and faster performance
)

### A. Create an endpoint

In [8]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

------------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

You can think of Jurassic-2 Mid as a smart auto-completion algorithm: give it some text as input and it will generate relevant text to naturally complete your input.

These two helpful concepts are worth being familiar with:
- **Prompt** - the input you provide to the model.
- **Completion** - the output text the model generates.

Enter a simple prompt: "To be, or", and let the model complete it

In [9]:
response = ai21.Completion.execute(destination=ai21.SageMakerDestination(endpoint_name),
                                   prompt="To be, or",
                                   maxTokens=4,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

 not to be: that is the question


As you can see, the model identifies the beginning of a famous quote, and completes it correctly.

### C. Prompt with instructions

**Why**? This model was specifically trained to follow natural language instructions. It is the most natural way to interact with large language models: simply tell the model what you want it to do, and it will follow.

**When?** Drafting, seeking for inspiration, or when the format and guidelines are "work in progress".

**How?** Just provide an instruction.

For this notebook, we will apply the model to creating a product description for an eCommerce site. We will start with providing the model a simple instruction.

In [10]:
instruction = """Write an engaging product description for clothing eCommerce site.
Product: Humor Men's Graphic T-Shirt.
Description:
"""

In [11]:
response = ai21.Completion.execute(destination=ai21.SageMakerDestination(endpoint_name),
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

This funny men's graphic t-shirt is perfect for any occasion. The shirt features an image of a cat wearing a crown, with the words "King of Cats" printed below. The shirt is made of high-quality cotton and is comfortable to wear.


#### Adjust the parameters
A useful parameter is the temperature. **You can increase creativity by tweaking the temperature.** With temperature 0, the model will always choose the most probable completion, so it will always be the same. Increasing the temperature will provide varying completions, where the completion may be different with every generation. We recommend starting with a value of 0.7.

In [18]:
response = ai21.Completion.execute(destination=ai21.SageMakerDestination(endpoint_name),
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3) # this will make the model generate 3 optional completions

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This funny t-shirt is perfect for any occasion. The graphic print is high-quality and sure to turn heads.
This funny t-shirt is perfect for men who love to laugh. The graphic on the front is eye-catching and sure to make people smile. The shirt is made of high-quality materials and is comfortable and stylish.
This funny men's graphic t-shirt is perfect for any occasion. The graphic features an image of a hipster cat holding a cup of coffee, and the text reads "Keep calm and cat on." The t-shirt is comfortable to wear and made of high-quality materials.


#### Personalize your prompt
The model can generate nice descriptions based on only an instruction, but they may not be aligned with your actual product. If you have special features, ask the model to include them:

In [23]:
detailed_instruction = """Write an engaging product description for a clothing eCommerce site. Make sure to include the following features in the description.
Product: Humor Men's Graphic T-Shirt.
Features:
- Soft cotton
- Short sleeve
- Have a print of Einstein's quote: "artificial intelligence is no match for natural stupidity”
Description:
"""

In [25]:
response = ai21.Completion.execute(destination=ai21.SageMakerDestination(endpoint_name),
                                   prompt=detailed_instruction,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This humorous men's T-shirt features a print of Albert Einstein's famous quote on artificial intelligence and natural stupidity. The shirt is made of soft cotton and is perfect for everyday wear.
This humorous men's graphic t-shirt is perfect for anyone who thinks artificial intelligence is no match for natural stupidity. The t-shirt is made from soft cotton and is comfortable to wear. The print is high quality and is sure to make you laugh.
This funny T-shirt is perfect for anyone who believes that artificial intelligence can't replace human intelligence. The soft cotton material will keep you comfortable, while the print of Einstein's famous quote is sure to turn heads.


### D. Prompt with examples

**Why?** Examples are helpful in assisting the model to comprehend and generate responses that adhere to the intended format.

**When?** Examples are particularly useful when there are stringent format constraints, a well-defined objective, and an overall structure to be maintained.

**How?** To establish a pattern for the model to follow, present a few instances (“shots”) of input-output pairs in the prompt. This enables the model to mimic the pattern. Then, provide the input for a query example and allow the model to generate a suitable completion. This approach is commonly referred to as a "*few-shot prompt*".

#### Create a few-shot prompt

We will build a few-shot prompt comprised of the following:

1. Prefix with 3 examples. Each example contains the relevant inputs (a product name and some features to incorporate) and the output (an engaging product description). They are separated by "##".

2. The query inputs. An unseen product name and set of features for which we would like the model to output a new product description. These should follow the same format of the inputs in the prefix.

First, we collect some example data for the prompt prefix:

In [26]:
EXAMPLES_DATA = [
    {"product": "Camp Director Crossbody Bag", 
     "features": ["Canvas purse", "Rainbow space print", "Leather trim", "Two securely-zipped compartments"], 
     "description": "This Camp Director Crossbody Bag is the perfect accessory for carrying your essentials in style. The canvas purse features a colorful space print with leather trim, and two securely-zipped compartments to keep your belongings safe. The adjustable strap makes it easy to wear over the shoulder or crossbody, and the purse is the perfect size for holding your phone, wallet, and more."},
    {"product": "Women's Boho Beach Dress", 
     "features": ["Midi dress", "Swing hem", "Slimming", "Special for summer"], 
     "description": "Women's Boho Beach Dress The style of midi dress with swing hem is slimming, It will make your tummy look flatter. This dress for women could not only make you feel cool, but also perfectly present the curve of female waist. Even in hot summer, it still keeps your elegant."},
    {"product":"Men's Soho Trousers", 
     "features": ["Designed by Retrolicious", "Stretch cotton fabric", "Side pockets", "Stripes print"],
     "description": "Designed with a side stripe and a modern slim fit, these Retrolicious Soho Trousers are the perfect pair for everyday wear. Its stretch cotton fabric makes it comfortable to wear all day long, while side pockets offer convenient storage."}
]

Then, we use the following helper functions to construct the prefix:

In [27]:
def list_features(features):
    return "\n".join(
        f"- {feat}" for feat in features
    )

def make_single_example(product, features, description):
    example = "Write an engaging product description for a clothing eCommerce site. Make sure to include the following features in the description.\n"
    example += f"Product: {product}\n"
    example += "Features:\n"
    example += list_features(features)
    example += "\nDescription:\n"
    example += description
    
    return example

SEPARATOR = "\n\n##\n\n"

FEW_SHOT_PREFIX = SEPARATOR.join(
    make_single_example(x["product"], x["features"], x["description"]) for x in EXAMPLES_DATA
)

And finally, we create a function to handle query inputs and create the full prompt:

In [28]:
def create_product_description_prompt(product, features):
    """
    Create a few-shot prompt to generate a product description with Jurassic-2 models given a name and features
    The prompt contains a preset sequence of examples followed by the query product name and features
    """
    return FEW_SHOT_PREFIX + SEPARATOR + make_single_example(product, features, '')  # keep the description blank and let the model generate

Let's see how this looks for the t-shirt with the specific features from before:

In [29]:
few_shot_prompt = create_product_description_prompt(
    product="Humor Men's Graphic T-Shirt", 
    features=["Soft cotton", "Short sleeve", "Have a print of Einstein's quote: \"artificial intelligence is no match for natural stupidity\""]
)

print(few_shot_prompt)

Write an engaging product description for a clothing eCommerce site. Make sure to include the following features in the description.
Product: Camp Director Crossbody Bag
Features:
- Canvas purse
- Rainbow space print
- Leather trim
- Two securely-zipped compartments
Description:
This Camp Director Crossbody Bag is the perfect accessory for carrying your essentials in style. The canvas purse features a colorful space print with leather trim, and two securely-zipped compartments to keep your belongings safe. The adjustable strap makes it easy to wear over the shoulder or crossbody, and the purse is the perfect size for holding your phone, wallet, and more.

##

Write an engaging product description for a clothing eCommerce site. Make sure to include the following features in the description.
Product: Women's Boho Beach Dress
Features:
- Midi dress
- Swing hem
- Slimming
- Special for summer
Description:
Women's Boho Beach Dress The style of midi dress with swing hem is slimming, It will 

In [30]:
response = ai21.Completion.execute(destination=ai21.SageMakerDestination(endpoint_name),
                                   prompt=few_shot_prompt,
                                   maxTokens=100,
                                   temperature=0.7,
                                   stopSequences=[SEPARATOR],
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This Humor Men's Graphic T-Shirt is perfect for anyone who believes in the intelligence of humans over artificial intelligence. The soft cotton material is comfortable, and the short sleeve makes it easy to wear. The print of Einstein's quote is eye-catching and sure to start a conversation.
This humorous men's graphic T-shirt is perfect for any Einstein fan. It features a print of his famous quote, "artificial intelligence is no match for natural stupidity." The shirt is made of soft cotton and comes in a variety of colors.
This Humor Men's Graphic T-Shirt is perfect for any man who doesn't take himself too seriously. The soft cotton material is comfortable to wear, and the short sleeve is perfect for warm summer days. The print of Einstein's quote is sure to get a laugh, and it's sure to become a favorite shirt in your wardrobe.


As you can see, the completions follow a similar pattern to the examples in the few-shot prompt.

### Interested in learning more?
Take a look at our [blog post](https://www.ai21.com/blog/building-cv-profile-generator-using-ai21-studio) to understand the process of building a good prompt.

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [31]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [32]:
model.delete_model()